# Explainability Analysis with SHAP and LIME

This notebook demonstrates:
1. SHAP explanations for models
2. LIME explanations for models
3. Interpretability metrics evaluation
4. Comparison of explanation methods

In [ ]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from utils.data_loader import DataLoader
from models import XGBoostClassifier, LightGBMClassifier, TransformerClassifier
from explainability import SHAPExplainer, LIMEExplainer
from metrics import InterpretabilityMetrics

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

%matplotlib inline

## Prepare Data and Train Model

In [ ]:
# Load data
dataset_name = 'breast_cancer'
loader = DataLoader(dataset_name, random_state=42)
X, y = loader.load_data()
data = loader.prepare_data(X, y, test_size=0.2, scale_features=True)

X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']
y_test = data['y_test']

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

In [ ]:
# Train XGBoost model
print("Training XGBoost...")
model = XGBoostClassifier(n_estimators=100, max_depth=6, random_state=42)
model.train(X_train, y_train)

# Evaluate
test_metrics = model.evaluate(X_test, y_test)
print("\nTest Metrics:")
for k, v in test_metrics.items():
    print(f"  {k}: {v:.4f}" if v is not None else f"  {k}: N/A")

## SHAP Explanations

In [ ]:
# Initialize SHAP explainer
print("Generating SHAP explanations...")
shap_explainer = SHAPExplainer(model, X_train, model_type='tree')
shap_values = shap_explainer.explain(X_test.head(100))

print(f"SHAP values shape: {np.array(shap_values).shape}")

In [ ]:
# SHAP summary plot
shap_explainer.plot_summary(X_test.head(100))
plt.show()

In [ ]:
# SHAP feature importance
shap_importance = shap_explainer.get_feature_importance(X_test.head(100))
print("\nTop 10 Features by SHAP Importance:")
print(shap_importance.head(10))

# Plot
plt.figure(figsize=(10, 8))
plt.barh(shap_importance['feature'][:20], shap_importance['importance'][:20])
plt.xlabel('Mean |SHAP value|')
plt.title('SHAP Feature Importance')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## LIME Explanations

In [ ]:
# Initialize LIME explainer
print("Generating LIME explanations...")
lime_explainer = LIMEExplainer(model, X_train, X_train.columns.tolist())

# Explain a single instance
instance_idx = 0
instance = X_test.iloc[instance_idx].values
exp = lime_explainer.explain_instance(instance, num_features=10)

print(f"\nLIME Explanation for instance {instance_idx}:")
print(f"Prediction: {model.predict(X_test.iloc[[instance_idx]])[0]}")
print(f"True label: {y_test.iloc[instance_idx]}")

In [ ]:
# Plot LIME explanation
lime_explainer.plot_explanation(instance, num_features=10)
plt.show()

In [ ]:
# LIME feature importance (aggregated)
lime_importance = lime_explainer.get_feature_importance(X_test, num_samples=50)
print("\nTop 10 Features by LIME Importance:")
print(lime_importance.head(10))

# Plot
plt.figure(figsize=(10, 8))
plt.barh(lime_importance['feature'][:20], lime_importance['importance'][:20])
plt.xlabel('Mean Importance Score')
plt.title('LIME Feature Importance (Aggregated)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## Compare SHAP and LIME

In [ ]:
# Compare feature importance rankings
comparison = pd.merge(
    shap_importance[['feature', 'importance']].rename(columns={'importance': 'SHAP'}),
    lime_importance[['feature', 'importance']].rename(columns={'importance': 'LIME'}),
    on='feature',
    how='outer'
).fillna(0)

# Normalize for comparison
comparison['SHAP_norm'] = comparison['SHAP'] / comparison['SHAP'].max()
comparison['LIME_norm'] = comparison['LIME'] / comparison['LIME'].max()

print("\nTop 15 Features - SHAP vs LIME Comparison:")
print(comparison.head(15).to_string(index=False))

In [ ]:
# Scatter plot comparison
plt.figure(figsize=(10, 8))
plt.scatter(comparison['SHAP_norm'], comparison['LIME_norm'], alpha=0.6)
plt.plot([0, 1], [0, 1], 'r--', label='Perfect Agreement')
plt.xlabel('SHAP Importance (normalized)')
plt.ylabel('LIME Importance (normalized)')
plt.title('SHAP vs LIME Feature Importance')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Interpretability Metrics

In [ ]:
# Prepare explanations for metrics
shap_explanations = []
lime_explanations = []

num_samples = min(50, len(X_test))
for i in range(num_samples):
    # SHAP
    shap_exp = shap_explainer.explain_instance(X_test.iloc[i])
    shap_explanations.append(shap_exp)
    
    # LIME
    lime_exp = lime_explainer.explain_instance(X_test.iloc[i].values, num_features=10)
    lime_exp_dict = dict(lime_exp.as_list())
    feature_exp = {}
    for feature in X_test.columns:
        for key, val in lime_exp_dict.items():
            if feature in key:
                feature_exp[feature] = val
                break
        if feature not in feature_exp:
            feature_exp[feature] = 0.0
    lime_explanations.append(feature_exp)

print(f"Generated {len(shap_explanations)} SHAP and {len(lime_explanations)} LIME explanations")

In [ ]:
# Calculate interpretability metrics
print("Calculating interpretability metrics...\n")

# Feature importance stability
importance_runs = [shap_importance, lime_importance]
stability = InterpretabilityMetrics.feature_importance_stability(importance_runs, method='spearman')
print(f"Feature Importance Stability (Spearman): {stability:.4f}")

# Explanation consistency
consistency = InterpretabilityMetrics.explanation_consistency(
    shap_explanations, lime_explanations, metric='cosine'
)
print(f"SHAP-LIME Explanation Consistency: {consistency:.4f}")

# Feature agreement
agreement_5 = InterpretabilityMetrics.feature_agreement(shap_importance, lime_importance, top_k=5)
agreement_10 = InterpretabilityMetrics.feature_agreement(shap_importance, lime_importance, top_k=10)
print(f"Feature Agreement (top-5): {agreement_5:.4f}")
print(f"Feature Agreement (top-10): {agreement_10:.4f}")

# Explanation fidelity
shap_fidelity = InterpretabilityMetrics.explanation_fidelity(
    model, X_test.head(num_samples), shap_explanations, top_k=5
)
lime_fidelity = InterpretabilityMetrics.explanation_fidelity(
    model, X_test.head(num_samples), lime_explanations, top_k=5
)
print(f"SHAP Explanation Fidelity: {shap_fidelity:.4f}")
print(f"LIME Explanation Fidelity: {lime_fidelity:.4f}")

# Explanation complexity
shap_complexity = InterpretabilityMetrics.explanation_complexity(shap_explanations)
lime_complexity = InterpretabilityMetrics.explanation_complexity(lime_explanations)
print(f"SHAP Explanation Complexity: {shap_complexity:.2f} features")
print(f"LIME Explanation Complexity: {lime_complexity:.2f} features")

In [ ]:
# Visualize metrics
metrics_data = {
    'Metric': ['Stability', 'Consistency', 'Agreement\n(top-5)', 'Agreement\n(top-10)'],
    'Score': [stability, consistency, agreement_5, agreement_10]
}

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Interpretability metrics
ax1.bar(metrics_data['Metric'], metrics_data['Score'])
ax1.set_ylabel('Score')
ax1.set_title('Interpretability Metrics')
ax1.set_ylim([0, 1])
ax1.grid(axis='y', alpha=0.3)

# Fidelity comparison
fidelity_data = ['SHAP', 'LIME']
fidelity_scores = [shap_fidelity, lime_fidelity]
ax2.bar(fidelity_data, fidelity_scores)
ax2.set_ylabel('Fidelity Score')
ax2.set_title('Explanation Fidelity')
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:
- Generation of SHAP explanations for tree-based models
- Generation of LIME explanations for any model
- Comparison of SHAP and LIME feature importance
- Calculation of rigorous interpretability metrics:
  - Feature importance stability
  - Explanation consistency
  - Feature agreement
  - Explanation fidelity
  - Explanation complexity

These metrics provide quantitative evaluation of model interpretability beyond just accuracy.